In [40]:
import sys
sys.path.append(r"D:\data analysis\code\WBI_analysis")  # 例如 r"C:\Users\YourName\Project"
import AnalysisMethod as am
import pandas as pd
import os
import numpy as np
import WBIFunctions as WBI
import cv2
from datetime import datetime
from sklearn.cluster import AgglomerativeClustering
from matplotlib.colors import ListedColormap, BoundaryNorm
from matplotlib.gridspec import GridSpec
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl

In [6]:
p_f_outside = r'Y:\\SZX\\2025_wbi_analysis\\good_WBI\\to_do_q'
f_folder = '20250716_lg9624-1gNa-002-24d-ov_006'
p_f = p_f_outside+'\\'+f_folder

# (与荧光对时的)行为数据导入，不需要重新分析处理的
'''激光段的行为数据,帧率为30Hz左右'''
mot_mid_vol = [f for f in os.listdir(p_f) if 'MotionMidlineMatchVol' in f][0]
if 'pkl' in mot_mid_vol:
    df_mot_vol = pd.read_pickle(os.path.join(p_f, mot_mid_vol)).reset_index(drop=True)
else:
    df_mot_vol = pd.read_csv(os.path.join(p_f, mot_mid_vol))

calcium_intensity= np.load(os.path.join(p_f, 'calcium_intensity.npy'))
mask = df_mot_vol['mask'].values.astype(bool)   # True 表示要置 NaN
calcium_intensity[:, mask] = np.nan
save_p = p_f.split('calcium_intensity.npy')[0]
print('文件大小:neuron*timestamp',calcium_intensity.shape)

文件大小:neuron*timestamp (117, 3387)


In [7]:
scale = 1.5
for i in range(calcium_intensity.shape[0]):
    row = calcium_intensity[i]
    mean_val = np.nanmean(row)
    row_no_nan = np.where(np.isnan(row), mean_val, row)
    smoothed = cv2.blur(row_no_nan.reshape(-1, 1), (7, 1)) * scale
    calcium_intensity[i] = smoothed[:, 0]
calcium_intensity[:, mask] = np.nan

In [ ]:
# 求前进速度和速率
df_mot_vol["head_velocity"] = df_mot_vol.apply(WBI.signed_norm, axis=1)
df_mot_vol["head_speed"] = df_mot_vol["head_velocity"].abs()
# 连续运动变量平滑
window_size = 15
# 计算移动平均值
df_mot_vol['sm_velocity'] = df_mot_vol['head_velocity'].rolling(window=window_size, min_periods=1).mean()
df_mot_vol['sm_speed'] = df_mot_vol['head_speed'].rolling(window=window_size, min_periods=1).mean()
# # 平滑ctx
window_size = 15
# 计算移动平均值
df_mot_vol['sm_CTX'] = df_mot_vol['CTX_left'].rolling(window=window_size, min_periods=1).mean()
df_mot_vol['sm_ang'] = df_mot_vol['ang_velocity'].rolling(window=window_size, min_periods=1).mean()


文件夹 'Y:\\SZX\\2025_wbi_analysis\\good_WBI\\to_do_q\20250716_lg9624-1gNa-002-24d-ov_006\2025-10-23-14_AnalysisFigs' 已创建！


In [ ]:

# 定义图片保存文件夹路径
now = datetime.now()
hour_str = now.strftime("%Y-%m-%d-%H")
folder_path = os.path.join(p_f, hour_str+'_AnalysisFigs')
# folder_path = os.path.join(p_f, 'AnalysisFigs')
# 检查文件夹是否存在
if not os.path.exists(folder_path):
    os.makedirs(folder_path)  # 创建文件夹
    print(f"文件夹 '{folder_path}' 已创建！")
else:
    print(f"文件夹 '{folder_path}' 已存在！")

In [15]:
signal_save_path = folder_path+'\\HierClustering'
os.makedirs(signal_save_path, exist_ok=True)

In [ ]:
choose_index = 0
thresh = 5
links=['ward','average','average','complete']
affs=['euclidean','cosine','cityblock','cosine']
vmin=-0.5
vmax=1
smooth_kernel = 10
print('calcium_intensity.shape',calcium_intensity.shape)

# 使用 np.corrcoef(calcium_intensity) 计算神经元钙信号的相关性矩阵
# 聚类：使用 am.cluster 和指定的链接方式 (link=links[choose_index]) 
# 和距离度量 (aff=affs[choose_index]) 对相关性矩阵进行聚类。
idx=am.cluster(np.corrcoef(calcium_intensity),link=links[choose_index],aff=affs[choose_index])
# idx为聚类之后的索引
bound=np.cumsum(am.GetBound(np.corrcoef(calcium_intensity),link=links[choose_index],aff=affs[choose_index],
                            threshold=thresh).astype(int))
# 调用 am.GetBound 计算矩阵分区的边界（bound），通过 np.cumsum 累积求和获取完整的边界数组
print('边界', bound)

calcium_intensity = calcium_intensity[idx]
calcium_intensity_smd = calcium_intensity.copy()


# 平滑数据
if smooth_kernel:
    for i,k in enumerate(calcium_intensity_smd):
        calcium_intensity_smd[i] = cv2.blur(k,(1,smooth_kernel))[:,0]
# 计算相关性矩阵
w_p2m = np.corrcoef(calcium_intensity_smd)

# 聚类并将聚类的结果画在相关性矩阵旁边
link = links[choose_index]
aff = affs[choose_index]
model = AgglomerativeClustering(distance_threshold=0, n_clusters=None,linkage=link, affinity=aff)
model = model.fit(w_p2m)
fig_h = int(10*round(calcium_intensity.shape[0]/10))
fig_w = int(0.7*round(calcium_intensity.shape[1]/100))
#     draw_calcium_curve(calcium_intensity,smooth_kernel=None,fig_size=(fig_w,fig_h),scale=1.5)

font_size = 100

'''输入事件开始index列表'''
neuron_ids = np.arange(calcium_intensity.shape[0])
col_draw = ['sm_velocity','sm_speed', 'sm_ang', 'sm_CTX','curvature', 'forward','turn_pc','turn_cor']

WBI.calcium_heatmap(calcium_intensity,df_mot_vol, col_draw,  neuron_ids,model,w_p2m, show_id_stride=10,
                show_vol_stride=500, heatmap_range=(0,0.6),wspace=0.06, hspace=0.2,bound_cluster=bound,
                unit_w=0.03, unit_h = 0.8, cal_height_ratio=30, smooth_kernel=smooth_kernel,
                font_size=font_size, font_color='black', idx = idx, vmin = vmin, vmax = vmax,
                    threshold=thresh, xlabel='Neuron Index',level=35, signal_save_path=signal_save_path)


calcium_intensity.shape (117, 3387)
边界 [  6  35  63  94 117]


In [34]:

'''注意这里是通过上一步排序后的钙信号数据！'''
calcium_intensity_sort = calcium_intensity.copy()
np.save(folder_path+'\\calcium_intensity_sorted.npy', calcium_intensity_sort)

In [17]:
block_path = signal_save_path+'\\BlockHighlight'
os.makedirs(block_path, exist_ok=True)
calcium_dict = {i: calcium_intensity_sort[i] for i in range(len(calcium_intensity_sort))}
WBI.block_visualize_traces(block_path,calcium_intensity, calcium_dict,df_mot_vol, bound, smooth=True, sigma=0.3,delta_y=1.2)

shape of calcium_traces (117, 3387)
分组是否和神经元数量相同： True


In [18]:
calcium_intensity_sort_T = calcium_intensity_sort.T
# 创建列名
col_neuron_names = [f"neuron{i+1}" for i in range(calcium_intensity_sort_T.shape[1])]
df_calcium = pd.DataFrame(calcium_intensity_sort_T, columns=col_neuron_names)
# 按照行索引合并
df_cal_motion = pd.concat([df_calcium, df_mot_vol], axis=1)

In [19]:
corr_folder = folder_path+'\\CorrAnalysis'
os.makedirs(corr_folder, exist_ok=True)
p_corr_ls = []  # 用于收集相关性分析的df的列表

In [20]:
# 根据静息处理运动参数列
# cols_ls = ['forward', "turn_pc", "turn_cor"]
cols_ls = ['forward']
df_cal_motion.loc[df_cal_motion['quies_pc']==1, cols_ls] = np.nan
cols_ls_2 = ['sm_velocity','sm_speed','sm_CTX']
df_cal_motion.loc[df_cal_motion['quies_pc']==1, cols_ls_2] = 0
cols_ls_3 = ['sm_ang','curvature']
df_cal_motion.loc[df_cal_motion['quies_pc']==1, cols_ls_3] = np.nan

In [21]:
# 前后后退转换
# 开始
t_win = 25
neu_cols = [col for col in df_cal_motion.columns if 'neuron' in col]
rev_start_idx = WBI.get_event_start_idx(df_cal_motion, 'forward', 'start')
df_rev_s_p = WBI.Paired_ttest_neu_discrete(df_cal_motion,'forward', rev_start_idx, neu_cols,
                                           test_window = t_win, event_note='RevStart')  # 注意帧率为3左右
if df_rev_s_p is not None:
    df_rev_s_c = WBI.multi_compar(df_rev_s_p,adj_method='fdr_bh')
    p_corr_ls.append(df_rev_s_c)
    sel_n_s = df_rev_s_c[(df_rev_s_c['p_cor']<=0.05)]['Neuron']
    rev_folder = corr_folder+'\\Reverse'
    WBI.plot_event_aligned_traces_combined(df_cal_motion,'forward',df_rev_s_c, sel_n_s, rev_start_idx,
                                   pre_window = t_win,post_window=t_win, 
                                       sort_by='pre',sort_window=t_win,  frame_interval=0.3,
                                       heatmap_range = (0,0.6),save_title_str='ReverseStart',x_label_str='Reverse Start',
                                       cmap='jet', unify_yaxis=True, save_path=rev_folder)


(事次*timestamp)(7, 51)
(事次*timestamp)(7, 50)
350
350
350
350


In [22]:

# 结束
t_win = 20
rev_end_idx = WBI.get_event_start_idx(df_cal_motion, 'forward', 'end')
df_rev_e_p = WBI.Paired_ttest_neu_discrete(df_cal_motion,'forward', rev_end_idx,
                                            neu_cols,test_window = t_win, event_note='RevEnd')  # 注意帧率为3左右
if df_rev_e_p is not None:
    df_rev_e_c = WBI.multi_compar(df_rev_e_p,adj_method='fdr_bh')
    p_corr_ls.append(df_rev_e_c)
    sel_n_e = df_rev_e_c[(df_rev_e_c['p_cor']<=0.1)]['Neuron']

    rev_folder = corr_folder+'\\Reverse'
    WBI.plot_event_aligned_traces_combined(df_cal_motion,'forward',df_rev_e_c, sel_n_e, rev_end_idx,
                                   pre_window = t_win,post_window=t_win, 
                                       sort_by='pre',sort_window=t_win,  frame_interval=0.3,
                                       heatmap_range = (0,0.6),save_title_str='ReverseEnd',x_label_str='Reverse End',
                                       cmap='jet', unify_yaxis=True, save_path=rev_folder)



1->0 转变的索引: [8, 347, 364, 403, 436, 527, 550, 883, 1183, 1191, 1199, 1760, 1787, 1858, 1872, 2052, 2089, 2119, 2129, 2146, 2167, 2754, 2776, 2794, 2807]
神经元0在开始索引为8事件前后window超出范围
(事次*timestamp)(3, 41)
神经元1在开始索引为8事件前后window超出范围
神经元2在开始索引为8事件前后window超出范围
神经元3在开始索引为8事件前后window超出范围
神经元4在开始索引为8事件前后window超出范围
神经元5在开始索引为8事件前后window超出范围
神经元6在开始索引为8事件前后window超出范围
神经元7在开始索引为8事件前后window超出范围
神经元8在开始索引为8事件前后window超出范围
神经元9在开始索引为8事件前后window超出范围
神经元10在开始索引为8事件前后window超出范围
神经元11在开始索引为8事件前后window超出范围
神经元12在开始索引为8事件前后window超出范围
神经元13在开始索引为8事件前后window超出范围
神经元14在开始索引为8事件前后window超出范围
神经元15在开始索引为8事件前后window超出范围
神经元16在开始索引为8事件前后window超出范围
神经元17在开始索引为8事件前后window超出范围
神经元18在开始索引为8事件前后window超出范围
神经元19在开始索引为8事件前后window超出范围
神经元20在开始索引为8事件前后window超出范围
神经元21在开始索引为8事件前后window超出范围
神经元22在开始索引为8事件前后window超出范围
神经元23在开始索引为8事件前后window超出范围
神经元24在开始索引为8事件前后window超出范围
神经元25在开始索引为8事件前后window超出范围
神经元26在开始索引为8事件前后window超出范围
神经元27在开始索引为8事件前后window超出范围
神经元28在开始索引为8事件前后window超出范围
神经元29在开始索引为8事件前后window超出范围
神经元30在开始索引为8事件前后window超出范

In [23]:
# omega turn(coiling开始)
# 开始
t_win = 10
# neu_cols = [col for col in df_cal_motion.columns if 'neuron' in col]
c_start_idx = WBI.get_event_start_idx(df_cal_motion, 'turn_pc', 'start')
df_turn_s_p = WBI.Paired_ttest_neu_discrete(df_cal_motion,'turn_pc', c_start_idx,
                                             neu_cols,test_window = t_win, event_note='CoilingStart')  # 注意帧率为3左右
if df_turn_s_p is not None:
    df_turn_s_c = WBI.multi_compar(df_turn_s_p,adj_method='fdr_bh')
    p_corr_ls.append(df_turn_s_c)
    sel_n_turn = df_turn_s_c[(df_turn_s_c['p_cor']<=0.05)]['Neuron']

    turn_folder = corr_folder+'\\OmegaTurn'
    WBI.plot_event_aligned_traces_combined(df_cal_motion,'turn_pc',df_turn_s_c, sel_n_turn, c_start_idx,
                                   pre_window = t_win,post_window=t_win, 
                                       sort_by='pre',sort_window=t_win,  frame_interval=0.3,
                                       heatmap_range = (0,0.6),save_title_str='CoilingStart',x_label_str='Coiling Start',
                                       cmap='jet', unify_yaxis=True, save_path=turn_folder)


(事次*timestamp)(5, 21)


In [24]:

# omega turn (校正后的)
# 开始
t_win = 10
oturn_start_idx = WBI.get_event_start_idx(df_cal_motion, 'turn_cor', 'start')
df_oturn_s_p = WBI.Paired_ttest_neu_discrete(df_cal_motion,'turn_cor', oturn_start_idx,
                                              neu_cols,test_window = t_win, event_note='OmegaStart')  # 注意帧率为3左右
if df_oturn_s_p is not None:
    df_oturn_s_c = WBI.multi_compar(df_oturn_s_p,adj_method='fdr_bh')
    p_corr_ls.append(df_oturn_s_c)
    sel_n_oturn = df_oturn_s_c[(df_oturn_s_c['p_cor']<=0.05)]['Neuron']

    turn_folder = corr_folder+'\\OmegaTurn'
    WBI.plot_event_aligned_traces_combined(df_cal_motion,'turn_cor',df_oturn_s_c, sel_n_oturn, oturn_start_idx,
                                   pre_window = t_win,post_window=t_win, 
                                       sort_by='pre',sort_window=t_win,  frame_interval=0.3,
                                       heatmap_range = (0,0.6),save_title_str='OmegaTurnStart',x_label_str='OmegaTurn Start',
                                       cmap='jet', unify_yaxis=True, save_path=turn_folder)


(事次*timestamp)(5, 21)


In [25]:
# omega turn end
# 开始
t_win = 10
oturn_end_idx = WBI.get_event_start_idx(df_cal_motion, 'turn_cor', 'end')
df_oturn_e_p = WBI.Paired_ttest_neu_discrete(df_cal_motion,'turn_cor', oturn_end_idx,
                                              neu_cols,test_window = t_win, event_note='OmegaEnd')  # 注意帧率为3左右
if df_oturn_e_p is not None:
    df_oturn_e_c = WBI.multi_compar(df_oturn_e_p,adj_method='fdr_bh')
    p_corr_ls.append(df_oturn_e_c)
    sel_n_oturn_e = df_oturn_e_c[(df_oturn_e_c['p_cor']<=0.05)]['Neuron']

    turn_folder = corr_folder+'\\OmegaTurn'
    WBI.plot_event_aligned_traces_combined(df_cal_motion,'turn_cor',df_oturn_e_c, sel_n_oturn_e, oturn_end_idx,
                                   pre_window = t_win,post_window=t_win, 
                                       sort_by='pre',sort_window=t_win,  frame_interval=0.3,
                                       heatmap_range = (0,0.6),save_title_str='OmegaTurnEnd',x_label_str='OmegaTurn End',
                                       cmap='jet', unify_yaxis=True, save_path=turn_folder)

1->0 转变的索引: [8, 379, 450, 893, 1199, 1877, 2813]
神经元0在开始索引为8事件前后window超出范围
(事次*timestamp)(5, 21)
神经元1在开始索引为8事件前后window超出范围
神经元2在开始索引为8事件前后window超出范围
神经元3在开始索引为8事件前后window超出范围
神经元4在开始索引为8事件前后window超出范围
神经元5在开始索引为8事件前后window超出范围
神经元6在开始索引为8事件前后window超出范围
神经元7在开始索引为8事件前后window超出范围
神经元8在开始索引为8事件前后window超出范围
神经元9在开始索引为8事件前后window超出范围
神经元10在开始索引为8事件前后window超出范围
神经元11在开始索引为8事件前后window超出范围
神经元12在开始索引为8事件前后window超出范围
神经元13在开始索引为8事件前后window超出范围
神经元14在开始索引为8事件前后window超出范围
神经元15在开始索引为8事件前后window超出范围
神经元16在开始索引为8事件前后window超出范围
神经元17在开始索引为8事件前后window超出范围
神经元18在开始索引为8事件前后window超出范围
神经元19在开始索引为8事件前后window超出范围
神经元20在开始索引为8事件前后window超出范围
神经元21在开始索引为8事件前后window超出范围
神经元22在开始索引为8事件前后window超出范围
神经元23在开始索引为8事件前后window超出范围
神经元24在开始索引为8事件前后window超出范围
神经元25在开始索引为8事件前后window超出范围
神经元26在开始索引为8事件前后window超出范围
神经元27在开始索引为8事件前后window超出范围
神经元28在开始索引为8事件前后window超出范围
神经元29在开始索引为8事件前后window超出范围
神经元30在开始索引为8事件前后window超出范围
神经元31在开始索引为8事件前后window超出范围
神经元32在开始索引为8事件前后window超出范围
神经元33在开始索引为8事件前后window超出范围
神经元34在开始索引为8事件前后windo

In [27]:
# 离散变量皮尔逊相关
beh_col = 'turn_pc'
p_uplim = 0.05
r_threshold = 0.25
df_neu_coil_p = WBI.corr_neu_con_var(df_cal_motion,behavior_col_ls=[beh_col],
                     neuron_col_str='neuron',adj_method = 'fdr_bh' )
df_neu_coil_p['sign'] = 0
df_neu_coil_p.loc[(df_neu_coil_p['p_cor']<=p_uplim)&(df_neu_coil_p['true_r'].abs()>r_threshold),'sign'] = 1
p_corr_ls.append(df_neu_coil_p)
# 显著的神经元
sign_neu_ls = df_neu_coil_p[(df_neu_coil_p['sign']==1)&((df_neu_coil_p['Event']==beh_col))]['Neuron'].unique()

hl_col_ls = ['forward',"turn_pc"]
WBI.plot_lines_neural_activity_disc(df_cal_motion, sign_neu_ls,df_neu_coil_p,beh_col,
                               hl_col =hl_col_ls,hl_color=[ "#1BA4FF","#DF6176",], num_bins=5,
                                 title=f"NeuCorrWith{beh_col.replace('sm_','')}HighlightedBy{'-'.join(hl_col_ls)}", color_map='inferno',
                               fs=25, x_bin=0.2, time_range_sel=[],axis_invisible = True,
                               color='#2878B5', x_label='', y_label='',sigma = 5,
                               strip_color='#9AC9DB', label_p = True,label_r = True, p_f=corr_folder+f'\\{beh_col}')

1015.8218374224061
总的时间范围：[42.40116257759385, 1058.223]


In [28]:
beh_col = 'turn_cor'
p_uplim = 0.05
r_threshold = 0.25
df_neu_omega_p = WBI.corr_neu_con_var(df_cal_motion,behavior_col_ls=[beh_col],
                     neuron_col_str='neuron',adj_method = 'fdr_bh' )
df_neu_omega_p['sign'] = 0
df_neu_omega_p.loc[(df_neu_omega_p['p_cor']<=p_uplim)&(df_neu_omega_p['true_r'].abs()>r_threshold),'sign'] = 1
p_corr_ls.append(df_neu_omega_p)
# 显著的神经元
sign_neu_ls = df_neu_omega_p[(df_neu_omega_p['sign']==1)&((df_neu_omega_p['Event']==beh_col))]['Neuron'].unique()

hl_col_ls = ['forward','turn_pc',"turn_cor"]
WBI.plot_lines_neural_activity_disc(df_cal_motion, sign_neu_ls,df_neu_omega_p,beh_col=beh_col,
                               hl_col =hl_col_ls,hl_color=[ "#1BA4FF","#F0EC61","#DF6176",], num_bins=5,
                                 title=f"NeuCorrWith{beh_col.replace('sm_','')}HighlightedBy{'-'.join(hl_col_ls)}", color_map='inferno',
                               fs=25, x_bin=0.2, time_range_sel=[],axis_invisible = True,
                               color='#2878B5', x_label='', y_label='',sigma = 5,
                               strip_color='#9AC9DB', label_p = True,label_r = True, p_f=corr_folder+f'\\{beh_col}')

1015.8218374224061
总的时间范围：[42.40116257759385, 1058.223]


In [29]:
# 离散变量皮尔逊相关
beh_col = 'forward'
p_uplim = 0.05
r_threshold = 0.25
df_neu_coil_p = WBI.corr_neu_con_var(df_cal_motion,behavior_col_ls=[beh_col],
                     neuron_col_str='neuron',adj_method = 'fdr_bh' )
df_neu_coil_p['sign'] = 0
df_neu_coil_p.loc[(df_neu_coil_p['p_cor']<=p_uplim)&(df_neu_coil_p['true_r'].abs()>r_threshold),'sign'] = 1
p_corr_ls.append(df_neu_coil_p)
# 显著的神经元
sign_neu_ls = df_neu_coil_p[(df_neu_coil_p['sign']==1)&((df_neu_coil_p['Event']==beh_col))]['Neuron'].unique()

hl_col_ls = ['forward']
WBI.plot_lines_neural_activity_disc(df_cal_motion, sign_neu_ls,df_neu_coil_p,beh_col,
                               hl_col =hl_col_ls,hl_color=[ "#1BA4FF"], num_bins=5,
                                 title=f"NeuCorrWith{beh_col.replace('sm_','')}HighlightedBy{'-'.join(hl_col_ls)}", color_map='inferno',
                               fs=25, x_bin=0.2, time_range_sel=[],axis_invisible = True,
                               color='#2878B5', x_label='', y_label='',sigma = 5,
                               strip_color='#9AC9DB', label_p = True,label_r = True, p_f=corr_folder+f'\\{beh_col}')

1015.8218374224061
总的时间范围：[42.40116257759385, 1058.223]


In [30]:
# 连续变量皮尔逊相关
df_neu_con_p = WBI.corr_neu_con_var(df_cal_motion,behavior_col_ls=['sm_velocity','sm_speed','sm_ang','curvature','sm_CTX'],
                     neuron_col_str='neuron',adj_method = 'fdr_bh' )
p_corr_ls.append(df_neu_con_p)
r_threhold = 0.2
x_ticklabels=[f"{i}" for i in range(1, 1+len(df_neu_con_p['Neuron'].unique()))]
WBI.plot_correlation_heatmap_nomask_from_df(df_neu_con_p,
                                 row_col = 'Event',
                                 col_col='Neuron',
                                     alpha_original=0.05,
                                     title="Neuron-Behavior Correlation Heatmap",
                                 mask_color='#f0f0f0',   # 灰白
                                 x_ticklabels=x_ticklabels,
                                 r_threshold=r_threhold,
                                 save_path=corr_folder,
                                 save_title='CorrNeuAllContinous',
                                    fs = 12.5)
beh_sig_ls = ['sm_velocity','sm_speed','sm_ang','curvature','sm_CTX']  # 可视化行为列表
# beh_sig_ls = ['sm_ang','curvature','sm_CTX']
for beh_col in beh_sig_ls:
    WBI.visualize_cont_beh_by_neu(df_neu_con_p,df_cal_motion, beh_col, p_uplim=0.05, r_threshold=0.2, p_f_con = corr_folder)    

图片已保存到: Y:\\SZX\\2025_wbi_analysis\\good_WBI\\to_do_q\20250716_lg9624-1gNa-002-24d-ov_006\2025-10-23-14_AnalysisFigs\CorrAnalysis
总的时间范围：[42.40116257759385, 1058.223]
总的时间范围：[42.40116257759385, 1058.223]
总的时间范围：[42.40116257759385, 1058.223]
总的时间范围：[42.40116257759385, 1058.223]
总的时间范围：[42.40116257759385, 1058.223]
总的时间范围：[42.40116257759385, 1058.223]
⚠️ 没有显著相关的神经元用于 sm_ang，跳过绘图
⚠️ 没有显著相关的神经元用于 sm_ang，跳过绘图
⚠️ 没有显著相关的神经元用于 sm_ang，跳过绘图
总的时间范围：[42.40116257759385, 1058.223]
总的时间范围：[42.40116257759385, 1058.223]
总的时间范围：[42.40116257759385, 1058.223]
总的时间范围：[42.40116257759385, 1058.223]
总的时间范围：[42.40116257759385, 1058.223]
总的时间范围：[42.40116257759385, 1058.223]


In [31]:
# 写出p值文件
df_corr_p = pd.concat(p_corr_ls, axis=0, ignore_index=True)
df_corr_p['File'] = f_folder
corr_p_path = corr_folder+f'\\{f_folder}_corr_p_cor.csv'
df_corr_p.to_csv(corr_p_path, index=False)

重新将显著性神经元可视化在热图上

In [63]:
def calcium_heatmap_sign(calcium_intensity,df, col_draw, suptitle, neuron_ids,model,w_p2m,
                     show_id_stride=20, show_vol_stride=10,
                    heatmap_range=(None,0.5),
                    unit_w=0.05, unit_h = 0.2, cal_height_ratio=20, wspace=0.125, hspace = 0.125
                    ,bound_cluster = [],smooth_kernel=15,
                    font_size=90, font_color='black',
                   idx=None,vmin=0,vmax=1,threshold=None,xlabel='',
                   cmap='jet',level=None,sig_matrix=None, signal_save_path=None, filename=''):
    '''
    calcium_intensity: （神经元个数 x 时间点）
    df： PCA和行为对齐
    col_draw: 需要画图的列名
    unit_w: 单位宽度(对应时间)
    unit_h: 单位高度（对应神经元数量）
    cal_height_ratio : 预期钙信号热图相对运动参数等条图的高度比例
    '''
    # 神经元数量及时间长度
    num_neurons, num_vols = calcium_intensity.shape
    
    
    if ('turn_cor' in col_draw)& ('turn_pc' in col_draw):
        # 两者画在一起
        turn_merge = True
        num_row = len(col_draw)
    else:
        turn_merge = False
        num_row = len(col_draw)+1
    
    height_ratios = [1.5 for i in range(num_row-1)]
    height_ratios.append(cal_height_ratio)
    num_sig_cols = len(sig_matrix.columns)
    gs = GridSpec(num_row, 4 + num_sig_cols, height_ratios=height_ratios,
                   width_ratios=[0.3,25,50,0.3] + [0.5]*num_sig_cols, wspace=wspace, hspace=hspace)
    # gs = GridSpec(num_row, 4, height_ratios=height_ratios, width_ratios=[0.3,25,50, 0.3], wspace=wspace, hspace=hspace)
    fig_h = (unit_h*num_neurons)/ cal_height_ratio *  sum(height_ratios)
    fig = plt.figure(figsize=(unit_w*num_vols+unit_h*num_neurons*1.1, fig_h))
    fig.suptitle(suptitle, fontsize=100, fontweight='bold')
    # 热图
    ax0 = fig.add_subplot(gs[-1, 1])
    # 树状图在上方
    ax1 = fig.add_subplot(gs[-4:-1, 1])
    
    
    
    # Override the default linewidth.
    mpl.rcParams['lines.linewidth'] = font_size*0.2
    am.plot_dendrogram(model, truncate_mode="level", p=level, \
                    no_labels=True, orientation='bottom', ax=ax1,color_threshold=threshold,
                   above_threshold_color='blue')
    ax1.set_xlim(ax1.get_xlim())
    # ax1.set_xlim([0.5,2])
    ax1.xaxis.set_ticks_position('none')  # 不显示x轴的刻度
    ax1.yaxis.set_ticks_position('none')  # 不显示y轴的刻度
    ax1.set_xticks([])  # 移除x轴的刻度标记
    ax1.set_yticks([])
    ax1.invert_yaxis()
    for spine in ax1.spines.values():
        spine.set_visible(False)
    ax_list=list(np.linspace(0,w_p2m.shape[0],24,dtype=int))
    # X_sort = w_p2m[idx][:,idx]
    X_sort = w_p2m.copy()
    # 重新排序之后画热图
    
    # 设置热图的corlorbar
    # [left, bottom, width, height]
    cbar_ax = fig.add_axes([0.0, 0.15, 0.01, 0.4])
    ax0=sns.heatmap(X_sort, ax=ax0, cmap=cmap, vmin=vmin, vmax=vmax,cbar=True, cbar_ax = cbar_ax, square=False)
    cbar_ax.yaxis.set_ticks_position("left")  # 将刻度放到左侧
    cbar_ax.yaxis.set_label_position("left")  # 将标签放到左侧
    cbar_ax.tick_params(labelsize=font_size, pad = font_size*0.75)
    cbar_ax.set_ylabel('Correlation', fontsize = font_size*1.25, labelpad = font_size*0.75)
    
    ax0.set_xticks(ax_list, ax_list,fontsize=font_size)
    ax0.set_yticks(ax_list, ax_list,fontsize=font_size)
    ax0.tick_params(axis = 'x', pad = font_size*0.75)
    ax0.tick_params(axis='y', labelrotation=0,pad = font_size*0.75)  # 将y轴标签设置为水平
    # 逆转y轴以对齐钙信号热图
    ax0.invert_yaxis()
    ax0.set_xlabel(xlabel,fontsize=font_size*1.25, labelpad = font_size*0.75)


    bound_pmd = bound_cluster
    bound_m1 = bound_cluster
    # 画边界
    for i in bound_pmd:
        ax0.axhline(y=i, color='white', linestyle='--')
    for i in bound_m1:
        ax0.axvline(x=i, color='white', linestyle='--')
    
    # 循环画运动参数
    for i, col in enumerate(col_draw):
        # 添加子图
        ax = fig.add_subplot(gs[i, 2])  # 从第一行开始
        vector = df[col].values
        if ('turn' not in col) & ('forward' not in col):
            heatmap_data = vector[np.newaxis, :]  # 变为 1 行 N 列
            im = ax.imshow(heatmap_data, cmap='jet', aspect='auto')  # 绘制热图

            # 调整 colorbar 参数
            cax = fig.add_subplot(gs[i, 3])  # colorbar 放在右侧
            cbar = plt.colorbar(im, cax=cax, fraction=0.5, orientation='vertical', aspect = 3)
            if 'smoothed_' in col:
                col = col.replace('smoothed_', '')
            cbar.ax.set_title(col, fontsize = font_size*0.85, pad = font_size*0.25 )
            cbar.ax.tick_params(labelsize=font_size*0.5, width=5, length=5, pad = font_size*0.25)  # 设置刻度大小和宽度
            # 设置标题和轴
            ax.set_xticks([])  # 隐藏x轴刻度
            ax.set_yticks([])  # 隐藏y轴刻度
        elif ('turn' in col) & (turn_merge==False):
            heatmap_data = vector[np.newaxis, :]  # 变为 1 行 N 列
            # Define two colors
            colors = [ 'grey','#FFC832']
            # Create a ListedColormap
            two_color_cmap = ListedColormap(colors)
            im = ax.imshow(heatmap_data, cmap=two_color_cmap, aspect='auto')  # 选择 colormap
            # 添加颜色条
            cax = fig.add_subplot(gs[i, 3])  # colorbar 放在右侧
            cbar = plt.colorbar(im, cax=cax, fraction=0.5, orientation='vertical', aspect = 3)  # 通过 plt.colorbar 添加颜色条
            cbar.ax.set_title(col, fontsize = font_size*0.85, pad = font_size*0.25 )
            ax.set_xticks([])  # 设置x轴刻度
            ax.set_yticks([])  # 隐藏y轴刻度
        elif 'forward_quies' == col:
            # 不只输出forward,将quies加上
            heatmap_data = vector[np.newaxis, :]  # 变为 1 行 N 列
            # Define two colors
            colors = [ 'red','blue','white']
            # Create a ListedColormap
            cmap = ListedColormap(colors)
            bounds = [-0.5, 0.5, 1.5, 2.5]
            norm = BoundaryNorm(bounds, cmap.N)
            im = ax.imshow(heatmap_data, cmap=cmap, norm=norm, aspect='auto')
            # 添加颜色条
            cax = fig.add_subplot(gs[i, 3])  # colorbar 放在右侧
            cbar = plt.colorbar(im, cax=cax, fraction=0.5, orientation='vertical', aspect = 3)  # 通过 plt.colorbar 添加颜色条
            cbar.ax.set_title(col, fontsize = font_size*0.85, pad = font_size*0.25 )
            ax.set_xticks([])  # 设置x轴刻度
            ax.set_yticks([])  # 隐藏y轴刻度
        elif 'forward' == col:
            heatmap_data = vector[np.newaxis, :]  # 变为 1 行 N 列
            # Define two colors
            colors = [ 'red','blue']
            # Create a ListedColormap
            two_color_cmap = ListedColormap(colors)
            im = ax.imshow(heatmap_data, cmap=two_color_cmap, aspect='auto')  # 选择 colormap
            # 添加颜色条
            cax = fig.add_subplot(gs[i, 3])  # colorbar 放在右侧
            cbar = plt.colorbar(im, cax=cax, fraction=0.5, orientation='vertical', aspect = 3)  # 通过 plt.colorbar 添加颜色条
            cbar.ax.set_title(col, fontsize = font_size*0.85, pad = font_size*0.25 )
            ax.set_xticks([])  # 设置x轴刻度
            ax.set_yticks([])  # 隐藏y轴刻度

    # 倒数第二行
    if turn_merge:
        ax = fig.add_subplot(gs[-2, 2])  # 第一行
        vector_pc = df['turn_pc'].values
        vector_cor = df['turn_cor'].values

        heatmap_data_pc = vector_pc[np.newaxis, :]
        heatmap_data_cor = vector_cor[np.newaxis, :]

        # --------- PC 层：灰色(0) + 黄色(1)
        color_pc = ['grey', '#FFC832']
        cmap_pc = ListedColormap(color_pc)
        ax.imshow(heatmap_data_pc, cmap=cmap_pc, aspect='auto')

        cax = fig.add_subplot(gs[-2, 3])  # colorbar 放在右侧
        cbar = plt.colorbar(im, cax=cax, fraction=0.5, orientation='vertical', aspect = 3)
        cbar.ax.set_title('coiling/turn', fontsize = font_size*0.85, pad = font_size*0.25 )
        cbar.ax.tick_params(labelsize=font_size*0.5, width=5, length=5, pad = font_size*0.25)

        # 去掉坐标
        ax.set_xticks([])
        ax.set_yticks([])
    
    
    # 绘制钙信号热力图 
    ax = fig.add_subplot(gs[-1, 2])  # 最后一行
    heatmap=sns.heatmap(calcium_intensity, vmin=heatmap_range[0], vmax=heatmap_range[1],
                        xticklabels=np.arange(num_vols)[::show_vol_stride],
                        yticklabels=neuron_ids[::show_id_stride],cbar=False,cmap='jet',
                         cbar_kws={'orientation':'horizontal'}, ax=ax)
    
    # 调整colorbar位置到heatmap下方
    fig = ax.get_figure()
    # 获取heatmap的位置信息
    pos = ax.get_position()

    # 将colorbar放在heatmap正下方，与其等宽
    cax = fig.add_axes([pos.x0, pos.y0 - 0.08, pos.width, 0.02])  # 在heatmap下方0.08的位置

    text_str = 'ΔR/R0'
    colorbar = plt.colorbar(heatmap.collections[0], cax=cax, orientation='horizontal')
    colorbar.ax.tick_params(labelsize=font_size, pad=font_size*0.75)
    colorbar.set_label(text_str, fontsize=font_size*1.25, labelpad=font_size*0.75)
    
    # # 竖直白线根据轨迹分区
    # if len(start_indices):
    #     x_sticks = start_indices[1:-1]
    #     for x in x_sticks:
    #         ax.axvline(x=x, color='white', linestyle='--', linewidth=font_size*0.12)  # Adjust color and linestyle as needed
    
    # 横向根据聚类结果分块
    if len(bound_cluster):
        for i in bound_cluster:
            ax.axhline(y=i, color='white', linestyle='--', linewidth=font_size*0.12)
    
    # ax.set_yticks(ticks=np.arange(0, num_neurons, show_id_stride), labels=neuron_ids[::show_id_stride],fontsize=font_size, 
    #               color=font_color)
#     ax.set_xticks(ticks=np.arange(0, num_vols, show_vol_stride), labels=np.arange(num_vols)[::show_vol_stride],fontsize=font_size, rotation=45, color=font_color)
    ax.set_xticks(ticks=np.arange(0, num_vols, show_vol_stride), labels=df.Vol_Time.astype(int).values[::show_vol_stride],
                  fontsize=font_size, rotation=0, color=font_color)
    ax.tick_params(pad = font_size*0.75)
    #     ax.set_xticks(ticks = df.Vol_Time
    # plt.xticks(ticks=np.arange(0, num_vols, show_vol_stride), labels=np.arange(0,1300,100),fontsize=font_size, rotation=45, color=font_color)
    # plt.title('Calcium activity traces (ΔR/R0) Heatmap',fontsize=font_size,)
    ax.set_xlabel('Time(s)',fontsize=font_size*1.25, color=font_color, labelpad = font_size*0.75)
#     ax.set_ylabel('Neuron Index',fontsize=font_size*1.25,color=font_color, labelpad = font_size*0.75)
    # plt.axis('off')
    # plt.gca().spines['left'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    # 设置y轴的刻度位置，使其从顶部开始，因为神经元通常从顶部向下绘制
    ax.invert_yaxis()
    ax.yaxis.tick_right()
    ax.yaxis.set_label_position('right')

    # 绘制显著性热图
    event_order_color = ['Velocity', 'Speed','CTX','Angular Velocity','Curvature','Forw-Rev','RevStart','RevEnd', 'Turn', 'TurnStart','TurnEnd']
    # 获取 colormap 对象（新 API）
    # cmap = plt.colormaps['tab20']  # 或 'viridis', 'Set3' 等
    # Tol Bright 12 colors (RGB from Paul Tol's scientific color schemes)
    tol_bright_12 = [
        "#332288",  # dark blue
        "#88CCEE",  # cyan
        "#44AA99",  # teal
        "#117733",  # green
        "#AA4499",  # purple
        "#6699CC",  # sky blue
        "#888888",  # gray
        "#661100",  # brown
        "#DDCC77",  # sand
        "#999933",  # olive
        "#882255",  
        "#CC6677",  # rose
    ]

    color_dict = {event: tol_bright_12[i % len(tol_bright_12)] for i, event in enumerate(event_order_color)}

    # 根据事件数量生成颜色字典
    # color_dict = {event: cmap(i / len(event_order_color)) for i, event in enumerate(event_order_color)}
    # 画显著性热图
    existing_columns = [col for col in event_order_color if col in sig_matrix.columns]
    sig_matrix_ordered = sig_matrix[existing_columns]
    for j, event in enumerate(sig_matrix_ordered.columns):
        ax_sig = fig.add_subplot(gs[-1, 3+j], sharey=ax)  # 在原 calcium heatmap 右侧追加
        data = sig_matrix_ordered[[event]].values  # 单列矩阵 (num_neurons, 1)
        cmap_sig = ListedColormap(['white', color_dict[event]])  # 白=不显著，红=显著
        
        sns.heatmap(
        data,
        cmap=cmap_sig,
        cbar=False,
        ax=ax_sig,
        vmin=0, vmax=1,        # 固定范围，防止自动缩放
        xticklabels=False,
        yticklabels=False,
        square=False
        )
        ax_sig.set_xticks([])
        # ax_sig.set_yticks([])
        ax_sig.invert_yaxis()
        ax_sig.set_xlabel(
            event, 
            fontsize=font_size * 0.7,   # 调小字号
            labelpad=font_size * 0.2,   # 与热图间距
            rotation=60,                # 旋转以防重叠
            ha='right'                  # 对齐方式
        )
        # 移除上方默认标题
        ax_sig.set_title("")
        
        # 去掉边框
        for spine in ax_sig.spines.values():
            spine.set_visible(False)
        # if j == len(sig_matrix.columns):
        #     ax_sig.set_yticks(np.linspace(0.5, len(data)-0.5, 5))  # 例如显示 5 个刻度
        #     ax_sig.set_yticklabels(
        #     np.linspace(1, len(data), 5, dtype=int),
        #     fontsize=font_size * 0.6
        #     )
        #     ax_sig.tick_params(axis='y', which='both', left=False, right=True)  # 刻度线放右侧
        # else:
        #     ax_sig.set_yticks([])

#     ax.gca().invert_yaxis()
#     plt.subplots_adjust(left=0.125, bottom=0.1, right=0.1, top=0.1, wspace=0.2, hspace=0.35)
    # plt.show()
    if smooth_kernel:
        plt.savefig(f'{signal_save_path}/{filename}clus_cal_heatmap(smooth_{smooth_kernel}).png', transparent=False,dpi=100)
        plt.close()
    else:
        plt.savefig(f'{signal_save_path}/{filename}clus_cal_heatmap.png', transparent=False,dpi=100)
        plt.close()

In [64]:
f_path = p_f
f = os.path.basename(f_path)
# 读取与荧光对齐的运动数据
file_df = [f for f in os.listdir(f_path+'\\') if '_MotionMidlineMatchVol.pkl' in f]
if not len(file_df):
    # raise FileNotFoundError("No folder containing 'AnalysisFigs' found.")
    pass
df_mot_vol = pd.read_pickle(os.path.join(f_path,file_df[0]))

# 读取重新排序后的荧光数据以及分析结果csv
fig_folder = [f for f in os.listdir(f_path) if 'AnalysisFigs' in f]
folders = [
    f for f in os.listdir(f_path)
    if 'AnalysisFigs' in f and os.path.isdir(os.path.join(f_path, f))
]

if not folders:
    raise FileNotFoundError("No folder containing 'AnalysisFigs' found.")
# 步骤2：获取每个文件夹的最后修改时间
folder_with_time = [
    (f, os.path.getmtime(os.path.join(f_path, f)))  # getmtime 返回时间戳
    for f in folders
]
# 步骤3：按修改时间降序排序，取最新的
latest_folder = max(folder_with_time, key=lambda x: x[1])[0]
save_path = f_path+'\\'+latest_folder
corr_path = f_path+'\\'+latest_folder+'\\CorrAnalysis\\'

p_val_path = [f for f in os.listdir(corr_path) if '_corr_p_cor.csv' in f][0]
df_p_cr = pd.read_csv(corr_path+'\\'+p_val_path)
df_p_cr['EventNote'] = df_p_cr['EventNote'].fillna(df_p_cr['Event'])
df_p_cr['sign'] = 0

turn_mask = (df_p_cr['Event'].isin(['turn_pc', 'turn_cor'])) & (df_p_cr['EventNote'].isna())
forward_mask = (df_p_cr['Event'] == 'forward') & (df_p_cr['EventNote'].isna())
df_p_cr['EventNote'] = np.where(turn_mask, 'Turn', 
                       np.where(forward_mask, 'Forw-Rev', df_p_cr['EventNote']))
# 这里比较奇怪，但是秉持着能用就行的原则，暂且靠两步修改
df_p_cr.loc[df_p_cr['EventNote']=='forward','EventNote'] = 'Forw-Rev'
df_p_cr.loc[df_p_cr['EventNote'].isin(['turn_pc','turn_cor']),'EventNote'] = 'Turn'

df_p_cr.loc[df_p_cr['EventNote']=='curvature','EventNote'] = 'Curvature'
# df_p_cr.loc[(df_p_cr['Event'].isin(['turn_pc', 'turn_cor'])) & 
#             (df_p_cr['EventNote'].isna()), 'EventNote'] = 'Turn'
# df_p_cr.loc[(df_p_cr['Event']=='forward') & (df_p_cr['EventNote'].isna()),'EventNote'] = 'Forw-Rev'

df_p_cr.loc[df_p_cr['EventNote']=='CoilingStart','EventNote'] = 'TurnStart'
df_p_cr.loc[df_p_cr['EventNote']=='sm_velocity','EventNote'] = 'Velocity'
df_p_cr.loc[df_p_cr['EventNote']=='sm_speed','EventNote'] = 'Speed'
df_p_cr.loc[df_p_cr['EventNote']=='sm_ang','EventNote'] = 'Angular Velocity'
df_p_cr.loc[df_p_cr['EventNote']=='sm_CTX','EventNote'] = 'CTX'
df_p_cr.loc[df_p_cr['EventNote']=='OmegaStart','EventNote'] = 'TurnStart'
df_p_cr.loc[df_p_cr['EventNote']=='OmegaEnd','EventNote'] = 'TurnEnd'
df_p_cr.loc[(df_p_cr['true_r'].abs()>=0.3) & (df_p_cr['p_cor']<=0.05) &
            (df_p_cr['EventNote'].notna()) &
            (df_p_cr['EventNote'].isin(['Velocity', 'Speed','CTX','Angular Velocity','Curvature','Turn','Forw-Rev'])), 'sign']  = 1
df_p_cr.loc[(df_p_cr['p_cor']<=0.05) &
            (df_p_cr['EventNote'].notna()) &
            (df_p_cr['EventNote'].isin(['RevStart','RevEnd', 'TurnStart','TurnEnd'])), 'sign']  = 1
# df_p_cr_sign = df_p_cr_sign[df_p_cr_sign['EventNote'] != 'OmegaStart']
# df_p_cr_sign = df_p_cr[df_p_cr['sign']==1]
# df_p_cr_sign = df_p_cr_sign[df_p_cr_sign['EventNote'] != 'turn_cor']

# 由于把turn_pc和turn_cor都合并了
sig_matrix = df_p_cr.pivot_table(
index='Neuron', 
columns='EventNote', 
values='sign', 
aggfunc='max',   # 或 'mean'，取最大值表示“只要有一次显著就记为显著”
fill_value=0
).reset_index()
sig_matrix['Neuron_num'] = sig_matrix['Neuron'].str.extract('(\d+)').astype(int)
# 按数字排序
sig_matrix = sig_matrix.sort_values('Neuron_num')
# 删除临时列
sig_matrix = sig_matrix.drop(columns=['Neuron_num','Neuron']).reset_index(drop=True)
# （可选）将 'Neuron' 设置为索引
# sig_matrix = sig_matrix.set_index('Neuron')
# sig_matrix = df_p_cr.pivot(index='Neuron', columns='EventNote', values='sign')


# 钙信号导入
calcium_intensity= np.load(os.path.join(f_path+'\\'+latest_folder, 'calcium_intensity_sorted.npy'))
# 根据mask列作预处理

mask = df_mot_vol['mask'].values.astype(bool)   # True 表示要置 NaN
calcium_intensity[:, mask] = np.nan


# save_p = p_f.split('calcium_intensity.npy')[0]
print('文件大小:neuron*timestamp',calcium_intensity.shape)
# 平滑calcium signal： 均值滤波器 (box filter),处理nan值
# scale = 1.5
# for i in range(calcium_intensity.shape[0]):
#     calcium_intensity[i] = (cv2.blur(calcium_intensity[i], (1, 7))*scale)[:,0]
scale = 1.5
for i in range(calcium_intensity.shape[0]):
    row = calcium_intensity[i]
    mean_val = np.nanmean(row)
    row_no_nan = np.where(np.isnan(row), mean_val, row)
    smoothed = cv2.blur(row_no_nan.reshape(-1, 1), (7, 1)) * scale
    calcium_intensity[i] = smoothed[:, 0]
calcium_intensity[:, mask] = np.nan
# 求前进速度和速率
df_mot_vol["head_velocity"] = df_mot_vol.apply(WBI.signed_norm, axis=1)
df_mot_vol["head_speed"] = df_mot_vol["head_velocity"].abs()
# 连续运动变量平滑
window_size = 15
# 计算移动平均值
df_mot_vol['sm_velocity'] = df_mot_vol['head_velocity'].rolling(window=window_size, min_periods=1).mean()
df_mot_vol['sm_speed'] = df_mot_vol['head_speed'].rolling(window=window_size, min_periods=1).mean()
# # 平滑ctx
window_size = 15
# 计算移动平均值
df_mot_vol['sm_CTX'] = df_mot_vol['CTX_left'].rolling(window=window_size, min_periods=1).mean()
df_mot_vol['sm_ang'] = df_mot_vol['ang_velocity'].rolling(window=window_size, min_periods=1).mean()

choose_index = 0
thresh = 5
links=['ward','average','average','complete']
affs=['euclidean','cosine','cityblock','cosine']
vmin=-0.5
vmax=1
smooth_kernel = 10
print('calcium_intensity.shape',calcium_intensity.shape)

valid_timepoints = ~mask
calcium_valid = calcium_intensity[:, valid_timepoints]
# 使用 np.corrcoef(calcium_intensity) 计算神经元钙信号的相关性矩阵
# 聚类：使用 am.cluster 和指定的链接方式 (link=links[choose_index]) 
# 和距离度量 (aff=affs[choose_index]) 对相关性矩阵进行聚类。
idx=am.cluster(np.corrcoef(calcium_valid),link=links[choose_index],aff=affs[choose_index])
print(idx)
# idx为聚类之后的索引
bound=np.cumsum(am.GetBound(np.corrcoef(calcium_valid),link=links[choose_index],aff=affs[choose_index],
                            threshold=thresh).astype(int))
# 调用 am.GetBound 计算矩阵分区的边界（bound），通过 np.cumsum 累积求和获取完整的边界数组
print('边界', bound)

calcium_valid = calcium_valid[idx]
calcium_intensity = calcium_intensity[idx]
# calcium_intensity_smd = calcium_intensity.copy()
# 平滑数据
if smooth_kernel:
    for i,k in enumerate(calcium_valid):
        calcium_valid[i] = cv2.blur(k,(1,smooth_kernel))[:,0]
# 计算相关性矩阵
w_p2m = np.corrcoef(calcium_valid)

# 聚类并将聚类的结果画在相关性矩阵旁边
link = links[choose_index]
aff = affs[choose_index]
model = AgglomerativeClustering(distance_threshold=0, n_clusters=None,linkage=link, affinity=aff)
model = model.fit(w_p2m)
font_size = 100

'''输入事件开始index列表'''
neuron_ids = np.arange(calcium_valid.shape[0])
df_mot_vol['forward_quies'] = df_mot_vol['forward'].copy()
df_mot_vol.loc[df_mot_vol['quies_pc'] == 1,'forward_quies'] = 2
col_draw = ['sm_velocity','sm_speed', 'sm_ang', 'sm_CTX','curvature', 'forward_quies','turn_pc','turn_cor']
# df_mot_valid = df_mot_vol[df_mot_vol['mask']==0]
calcium_heatmap_sign(calcium_intensity,df_mot_vol, col_draw, f,neuron_ids,model,w_p2m, show_id_stride=10,
                show_vol_stride=500, heatmap_range=(0,0.6),wspace=0.06, hspace=0.2,bound_cluster=bound,
                unit_w=0.03, unit_h = 0.8, cal_height_ratio=30, smooth_kernel=smooth_kernel,
                font_size=font_size, font_color='black', idx = idx, vmin = vmin, vmax = vmax,
                    threshold=thresh, xlabel='Neuron Index',level=35,sig_matrix = sig_matrix, signal_save_path=save_path, filename=f)

文件大小:neuron*timestamp (117, 3387)
calcium_intensity.shape (117, 3387)
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116]
边界 [  6  35  63  94 117]
